## CUSTOMER LIFETIME VALUE

CLTV = (Customer Value / Churn Rate) * Profit Margin 

-churn_rate = 1 - repeat_rate

-customer_value = average_order_value * purchase_frequency

-average_order_value = total_price / total_transaction

-purchase_frequency = total_transaction / total_number_of_customers

-repeat_rate = (Number of customers who make multiple purchases) / total_number_of_customers

-profit_margin = total_price * 0.10

In [1]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
pd.set_option("display.max_columns", None)
pd.set_option("display.float_format", lambda x: "%.5f" % x)
pd.set_option("display.expand_frame_repr", False)

df_ = pd.read_excel("/Users/betulyilmaz/Desktop/Miuul/CRM Analytics/Datasets/online_retail_II.xlsx")
df = df_.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95000,13085.00000,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75000,13085.00000,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10000,13085.00000,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25000,13085.00000,United Kingdom


In [2]:
def check_df(dataframe):
    print('------------ Shape ------------')
    print(dataframe.shape)
    print('------------ Types ------------')
    print(dataframe.dtypes)
    print('------------ Describe ------------')
    print(dataframe.describe().T)
    print('------------ NA ------------')
    print(dataframe.isnull().sum())

check_df(df)

------------ Shape ------------
(525461, 8)
------------ Types ------------
Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object
------------ Describe ------------
                   count                           mean                  min                  25%                  50%                  75%                  max        std
Quantity    525461.00000                       10.33767          -9600.00000              1.00000              3.00000             10.00000          19152.00000  107.42411
InvoiceDate       525461  2010-06-28 11:37:36.845017856  2009-12-01 07:45:00  2010-03-21 12:20:00  2010-07-06 09:51:00  2010-10-15 12:45:00  2010-12-09 20:01:00        NaN
Price       525461.00000                        4.68883         -53594.36000              1.25000              2.10000 

In [3]:
df = df[(df['Quantity'] > 0)]
df.dropna(inplace=True)
df = df[~df["Invoice"].astype("str").str.contains("C", na=False)]

check_df(df)

------------ Shape ------------
(407695, 8)
------------ Types ------------
Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
dtype: object
------------ Describe ------------
                   count                           mean                  min                  25%                  50%                  75%                  max        std
Quantity    407695.00000                       13.58669              1.00000              2.00000              5.00000             12.00000          19152.00000   96.84223
InvoiceDate       407695  2010-07-01 10:10:10.782177792  2009-12-01 07:45:00  2010-03-26 14:01:00  2010-07-09 15:46:00  2010-10-14 17:09:00  2010-12-09 20:01:00        NaN
Price       407695.00000                        3.29419              0.00000              1.25000              1.95000 

In [4]:
df["TotalPrice"] = df["Quantity"] * df["Price"]

cltv_c = df.groupby("Customer ID").agg({"Invoice": lambda x: x.nunique(),
                                        "Quantity": lambda x: x.sum(),
                                        "TotalPrice": lambda x: x.sum()})

cltv_c.columns = ["total_transaction", "total_unit", "total_price"]
cltv_c.head()

,total_transaction,total_unit,total_price
Customer ID,,,
12346.00000,11,70,372.86000
12347.00000,2,828,1323.32000
12348.00000,1,373,222.16000
12349.00000,3,993,2671.14000
12351.00000,1,261,300.93000


In [5]:
# Average Order Value (total_price / total_transaction) -islem basina ort harcama 

cltv_c["average_order_value"] = cltv_c["total_price"] / cltv_c["total_transaction"]

In [6]:
# Purchase Frequency (total_transaction / total_number_of_customers) -harcama sıklığı

cltv_c["purchase_frequency"] = cltv_c["total_transaction"] / cltv_c.shape[0]

In [7]:
# Repeat Rate (Number of customers who make multiple purchases) / total_number_of_customers

repeat_rate = cltv_c[cltv_c["total_transaction"] > 1].shape[0] / cltv_c.shape[0]

In [8]:
# Churn Rate (1 - repeat_rate)

churn_rate = 1 - repeat_rate

In [9]:
# Profit Margin (total_price * 0.10)

cltv_c["profit_margin"] = cltv_c["total_price"] * 0.10

In [10]:
# Customer Value (average_order_value * purchase_frequency)

cltv_c["customer_value"] = cltv_c["average_order_value"] * cltv_c["purchase_frequency"]

In [11]:
# Customer Lifetime Value ((customer_value / churn_rate) x profit_margin)

cltv_c["cltv"] = (cltv_c["customer_value"] / churn_rate) * cltv_c["profit_margin"]

cltv_c.sort_values(by="cltv", ascending=False).head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv
Customer ID,,,,,,,,
18102.00000,89,124216,349164.35000,3923.19494,0.02063,34916.43500,80.93749,8579573.77276
14646.00000,78,170342,248396.50000,3184.57051,0.01808,24839.65000,57.57916,4342070.45829
14156.00000,102,108107,196566.74000,1927.12490,0.02364,19656.67400,45.56484,2719105.08615
14911.00000,205,69722,152147.57000,742.18327,0.04752,15214.75700,35.26833,1629055.80978
13694.00000,94,125893,131443.19000,1398.33181,0.02179,13144.31900,30.46898,1215855.89003


In [12]:
# Segmentation

cltv_c["segment"] = pd.qcut(cltv_c["cltv"], 4, labels=["D", "C", "B", "A"])

cltv_c.sort_values(by="cltv", ascending=False).head()

,total_transaction,total_unit,total_price,average_order_value,purchase_frequency,profit_margin,customer_value,cltv,segment
Customer ID,,,,,,,,,
18102.00000,89,124216,349164.35000,3923.19494,0.02063,34916.43500,80.93749,8579573.77276,A
14646.00000,78,170342,248396.50000,3184.57051,0.01808,24839.65000,57.57916,4342070.45829,A
14156.00000,102,108107,196566.74000,1927.12490,0.02364,19656.67400,45.56484,2719105.08615,A
14911.00000,205,69722,152147.57000,742.18327,0.04752,15214.75700,35.26833,1629055.80978,A
13694.00000,94,125893,131443.19000,1398.33181,0.02179,13144.31900,30.46898,1215855.89003,A


In [13]:
cltv_c.groupby("segment").agg({"count", "mean", "sum"})

total_transaction              total_unit                total_price                     average_order_value                    purchase_frequency               profit_margin                    customer_value                         cltv                     
                     mean count    sum       mean count      sum        mean count           sum                mean count          sum               mean count     sum          mean count          sum           mean count        sum        mean count            sum
segment                                                                                                                                                                                                                                                                   
D                 1.22892  1079   1326  109.00463  1079   117616   178.18826  1079  192265.13000           157.37300  1079 169805.46550            0.00028  1079 0.30737      17.81883  1079  19226.51300        0.04130  1079   44.56772     2.64071  1079     2849.32790
C                 2.00371  1078   2160  283.05659  1078   305135   475.89652  1078  513016.45300           294.45487  1078 317422.35514            0.00046  1078 0.50070      47.58965  1078  51301.64530        0.11031  1078  118.91897    16.86852  1078    18184.26342
B                 3.76902  1078   4063  680.15863  1078   733211  1131.35918  1078 1219605.20000           389.72832  1078 420127.12365            0.00087  1078 0.94182     113.13592  1078 121960.52000        0.26225  1078  282.70867    96.05739  1078   103549.86329
A                10.81186  1079  11666 4062.33735  1079  4383262  6401.40546  1079 6907116.49100           670.96592  1079 723972.22347            0.00251  1079 2.70422     640.14055  1079 690711.64910        1.48387  1079 1601.09330 23408.05843  1079 25257295.04118